## Coleta dos dados utilizando BeautifulSoup

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

*get_product_info* é uma função que coleta as informações (sku, title, brand, model, price, category) de um produto da página do Magazine Luiza. Quando uma informação não é encontrada na página, o valor é preenchido como vazio.

In [37]:
def get_product_info(url, ctr):
    r = requests.get(url)

    html_doc = r.text

    soup = BeautifulSoup(html_doc, 'html.parser')

    sku = ''
    # SKU (código do produto na loja)
    span = soup.select('small.header-product__code')
    if(len(span) > 0):
        span = span[0]
        sku = span.select('meta')
        if(len(sku) > 0):
            sku = sku[0].attrs['content']
            sku = sku.split(':')
            sku = sku[len(sku) - 1]
        
        # Titulo
    title = soup.title.string

    # soup.select('table.description__box')
    table = soup.select('table.description__box--wildSand')
    brand = ''
    model = ''
    price = ''
    if(len(table) > 0 ):
        table = table[0]
        t = table.select('td.description__information-right')[0]
        boxes = t.select('table.description__box')
        for box in boxes:
            # Marca
            if(box.select('td.description__information-box-left')[0].text.strip() == 'Marca'):
                brand = box.select('td.description__information-box-right')[0].text.strip()

            # Modelo
            if(box.select('td.description__information-box-left')[0].text.strip() == 'Modelo'):
                model = box.select('td.description__information-box-right')[0].text.strip()

    # Preço
    price = soup.select('span.price-template__text')
    if(len(price) > 0):
        price = price[0].text
    else:
        price = -1
    # Categoria
    category = ctr

    return (sku, title, brand, model, price, category)

*get_sub_urls* é uma função que a partir da página de produtos de uma categoria, obtem a url de todos produtos para aquela categoria, criando um dicionário com a url e a categoria do produto.

In [3]:
def get_sub_urls(url_main):
    # saída por produtos.
    hrefs = np.empty(0)
    size = -1
    id = 0
    cat = url_main.split('/')[3]

    while(hrefs.size - size> 0):
        size = hrefs.size
        # Id da página.
        id = id + 1
 
        # Página de produtos.
        urlf = url_main+str(id)+'/'
        print("getting... " + urlf)
        # Obtendo link para cada produto.
        r = requests.get(urlf)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        prods = soup.select('li.product')
        for prod in prods: 
            a = prod.select('a')[0]
            href = {
                'href': a.attrs['href'],
                'cat': cat
            }
            hrefs= np.append(hrefs, href)
        
    return hrefs

In [8]:
url = 'https://www.magazineluiza.com.br/geladeira-refrigerador-consul-frost-free-duplex-437l-bem-estar-crm55abana-branco/p/0888942/ed/elgf/'
type = 'geladeira-refrigerador'
get_product_info(url, type)

sku:088894200


('088894200',
 'Geladeira/Refrigerador Consul Frost Free Duplex - 437L Bem Estar CRM55ABANA Branco - Geladeira/Refrigerador Frost Free - Magazine Luiza',
 'Consul',
 'CRM55A',
 '2.137,41',
 'geladeira-refrigerador')

In [5]:
url = "https://www.magazineluiza.com.br/lava-e-seca-lg-11kg-smart-care-prime-touch-14-programas-de-lavagem-agua-quente/p/0105417/ed/ela1/"
type = 'lavadora-lava-e-seca'
get_product_info(url, type)

sku:010541701


('010541701',
 'Lava e Seca LG 11kg Smart Care Prime Touch  - 14 Programas de Lavagem Água Quente - Lavadora de Roupas Lava e Seca - Magazine Luiza',
 'LG',
 'Prime Touch',
 '2.450,91',
 'lavadora-lava-e-seca')

Coleta das urls dos produtos a partir das duas urls de categorias fornecidas.

In [6]:
product_list = ["https://www.magazineluiza.com.br/lavadora-de-roupas-lava-e-seca/eletrodomesticos/s/ed/ela1/", "https://www.magazineluiza.com.br/geladeira-refrigerador/eletrodomesticos/s/ed/refr/"]


urls = np.empty(0)
for url in product_list:
    urls = np.append(urls, get_sub_urls(url))

print(urls.shape)


getting... https://www.magazineluiza.com.br/lavadora-de-roupas-lava-e-seca/eletrodomesticos/s/ed/ela1/1/
getting... https://www.magazineluiza.com.br/lavadora-de-roupas-lava-e-seca/eletrodomesticos/s/ed/ela1/2/
getting... https://www.magazineluiza.com.br/lavadora-de-roupas-lava-e-seca/eletrodomesticos/s/ed/ela1/3/
getting... https://www.magazineluiza.com.br/lavadora-de-roupas-lava-e-seca/eletrodomesticos/s/ed/ela1/4/
getting... https://www.magazineluiza.com.br/lavadora-de-roupas-lava-e-seca/eletrodomesticos/s/ed/ela1/5/
getting... https://www.magazineluiza.com.br/lavadora-de-roupas-lava-e-seca/eletrodomesticos/s/ed/ela1/6/
getting... https://www.magazineluiza.com.br/geladeira-refrigerador/eletrodomesticos/s/ed/refr/1/
getting... https://www.magazineluiza.com.br/geladeira-refrigerador/eletrodomesticos/s/ed/refr/2/
getting... https://www.magazineluiza.com.br/geladeira-refrigerador/eletrodomesticos/s/ed/refr/3/
getting... https://www.magazineluiza.com.br/geladeira-refrigerador/eletrodomest

A partir das urls coletadas, realiza a coleta da base de produtos.

In [85]:
products = np.empty((0,6))
for url in urls:
    prod = np.array(get_product_info(url['href'], url['cat'])).reshape((1,6))
    products = np.append(products, prod, axis=0)

Salvando os dados coletados em um csv utilizando pandas.

In [ ]:
df = pd.DataFrame()
# return (sku, title, brand, model, price, category)
df['sku'] = products[:, 0]
df['title'] = products[:, 1]
df['brand'] = products[:, 2]
df['model'] = products[:, 3]
df['price'] = products[:, 4]
df['category'] = products[:, 5]
df.to_csv('magazine_data.csv', index=True)

## Analise dos dados utilizando BagOfWords e Naive Bayes.

A partir dos dados coletados, foi criado um modelo de classificação para predizer a categoria do produto a partir do seu título.
Foi utilizado 70% dos dados para treinar o modelo (conjunto de treino) e os 30% restantes para avaliar (conjunto de teste).

O modelo escolhido foi o Naive Bayes com BagOfWords.

In [118]:
# Lendo dados
df = pd.read_csv('magazine_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] # Removendo Index Unnamed
df.head()

,sku,title,brand,model,price,category
0,10541701,Lava e Seca LG 11kg Smart Care Prime Touch - ...,LG,Prime Touch,"2.450,91",lavadora-de-roupas-lava-e-seca
1,215900400,"Lava e Seca Samsung 10,2kg Inox WD6000 - 13 Pr...",Samsung,WD6000,"2.868,91",lavadora-de-roupas-lava-e-seca
2,217304200,"Lava e Seca LG 8,5kg Mega Touch - 9 Programas ...",LG,Mega Touch,"2.299,90",lavadora-de-roupas-lava-e-seca
3,10541600,Lava e Seca LG 11kg Smart Care Prime Touch - ...,LG,Prime Touch,"2.979,90",lavadora-de-roupas-lava-e-seca
4,219008900,"Lava e Seca Samsung 10,2kg Prata WD4000 - 12 P...",Samsung,WD4000,"2.422,41",lavadora-de-roupas-lava-e-seca


In [114]:
#  Realizando pré processamento dos dados.

# Convertendo para lower case
df['title'] = df['title'].str.lower()

target_names = df['category'].astype('category').cat.categories

# Convertendo as categorias para numérico
df['category'] = df['category'].astype('category').cat.codes

In [115]:
# Separando conjunto de treino (70%) e conjunto de teste (30%)
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.30, random_state=42, shuffle=True)

# Conjunto de treino.
x_train = train['title']
y_train = train['category']

# Conjunto de testes.
x_test = test['title']
y_test = test['category']

In [116]:
# Construindo modelo para a classificação.
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
x_count_train = count_vect.fit_transform(x_train)


from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_count_train, y_train)

In [117]:
# Avaliando modelo construido.
x_count_test = count_vect.transform(x_test)

predicted = clf.predict(x_count_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, predicted, target_names=target_names))


                                precision    recall  f1-score   support

        geladeira-refrigerador       1.00      1.00      1.00        90
lavadora-de-roupas-lava-e-seca       1.00      1.00      1.00        75

                   avg / total       1.00      1.00      1.00       165

